In [84]:
import os
import pandas as pd

In [85]:

DOWNLOAD_DIR = "~/Downloads"

def get_xls_paths():
    path_str = os.path.expanduser(DOWNLOAD_DIR)
    file_names = os.listdir(path_str)
    result = []
    for file_name in file_names:
        if "지역_위치별(주유소)" in file_name:
            result.append(f'{path_str}/{file_name}')
    return result

def concat_xls_data(file_paths):
    dfs = []
    for file_path in file_paths:
        target = pd.read_excel(file_path, header=2, index_col=1, na_values='-')
        dfs.append(target)
    return pd.concat(dfs)

def create_get_dir(file_name, save_dir):
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    return os.path.join(save_dir, file_name) 

def get_urb(address):
    return address.split(" ")[1]

In [86]:
file_paths = get_xls_paths()
base = concat_xls_data(file_paths)
base_csv_dir = create_get_dir('base.csv', '../data')
base.to_csv(base_csv_dir)

In [87]:
processed = base \
    .copy() \
    .drop(columns=["지역", "주소", "전화번호"])
processed.insert(1, "자치구", [get_urb(d) for d in base["주소"]])
processed_csv_dir = create_get_dir('processed.csv', '../data')
processed.to_csv(processed_csv_dir)